# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Dany's Directory
%cd /content/drive/My Drive/Colab Notebooks/imagenet/

/content/drive/My Drive/Colab Notebooks/imagenet


In [ ]:
# Christopher's directory
%cd /content/drive/My Drive/imagenet/

/content/drive/.shortcut-targets-by-id/1cZ-V95iJX2lZXKQeHjcbONaKRVlwKagP/imagenet


# Import Libraries


In [3]:
import os
import cv2
import shutil
import random
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
!pip install tensorflow --q

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

#  Moving the Data


In [5]:
# Sample images and move them to the proper directories
random.seed(42)
src_path = os.getcwd()

# Inspecting the Data

In [6]:
training_path = os.getcwd() + '/preliminary_model_testing_data/train'
validation_path = os.getcwd() + '/preliminary_model_testing_data/validation'
testing_path = os.getcwd() + '/preliminary_model_testing_data/test'

In [7]:
# Create Image Data Generator instance
data_generator = ImageDataGenerator() # rescale=1.0/255.0, featurewise_center=True, horizontal_flip=True, brightness_range=[0.7,1.3], zoom_range=[0.7, 1.3], shear_range = 0.2,

# Create training iterator, validation iterator & testing iterator
train_it = data_generator.flow_from_directory(training_path,
                                              class_mode="binary",
                                              batch_size=32,
                                              target_size = (224, 224))

validation_it = data_generator.flow_from_directory(validation_path,
                                                   class_mode="binary",
                                                   batch_size=16,
                                                   target_size = (224, 224))

test_it = data_generator.flow_from_directory(testing_path,
                                             class_mode="binary",
                                             batch_size=16,
                                             target_size = (224, 224),)

batchX, batchY = train_it.next()
print("BatchX shape: {}, min: {}, max: {}".format(batchX.shape, batchX.min(), batchX.max()))
print("BatchY shape: {}".format(batchY.shape))

Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
BatchX shape: (32, 224, 224, 3), min: 0.0, max: 255.0
BatchY shape: (32,)


# Resnet 101

In [ ]:
# Re-size all the images to this
IMAGE_SIZE = [224, 224] # The ResNet50 model was originally trained on images of this size from the ImageNet dataset.

In [ ]:
resnet = ResNet101(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False) # include_top=False indicates that the final classification layer of the original ResNet50 model should not be included because the original ResNet50 model was trained to classify images into 1000 different classes from the ImageNet dataset. So, we need to add our own custom classification layer(s) later.

171446536/171446536 [==============================] - 2s 0us/step


In [ ]:
for layer in resnet.layers:
    layer.trainable = False # freeze the weights of all layers in the ResNet50 base model, making them non-trainable during the training process.

In [ ]:
x = Flatten()(resnet.output)
# Most neural network architectures, including ResNet50, use convolutional and pooling layers to extract spatial features from the input images. The output of these layers is typically a multi-dimensional tensor (e.g., (batch_size, height, width, channels)). However, dense layers expect a one-dimensional vector as input.
# The purpose of flattening the output tensor is to prepare it for the following dense (fully connected) layers that we will add for our binary classification task.

In [ ]:
prediction = Dense(1, activation='sigmoid')(x) # creating a dense (fully connected) layer as the final output layer for our classification task.

In [ ]:
# Create Model
# model = Model(inputs=resnet.input, outputs=prediction) # creating the final model architecture by combining the ResNet50 base model with the new classification layer we added
model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
# Compile Model
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Callbacks
callbacks = [
    ModelCheckpoint("Models/PreliminaryTesting/ResNet101_no_rescaling_model.h5", save_weights_only=True, save_freq=5)
]

# Fit the model with early stopping callback
history = model.fit_generator(
  train_it,
  validation_data=validation_it,
  epochs=20,
  steps_per_epoch=10,
  validation_steps=5,
  verbose=1,
  callbacks=callbacks
)

<ipython-input-16-15e514925125>:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20
10/10 [==============================] - 233s 18s/step - loss: 5.0962 - accuracy: 0.4406 - val_loss: 3.0727 - val_accuracy: 0.6000
Epoch 2/20
10/10 [==============================] - 182s 18s/step - loss: 3.6620 - accuracy: 0.6500 - val_loss: 2.6788 - val_accuracy: 0.6125
Epoch 3/20
10/10 [==============================] - 171s 17s/step - loss: 2.8764 - accuracy: 0.7063 - val_loss: 4.2684 - val_accuracy: 0.6625
Epoch 4/20
10/10 [==============================] - 169s 17s/step - loss: 2.9006 - accuracy: 0.7250 - val_loss: 2.9968 - val_accuracy: 0.6375
Epoch 5/20
10/10 [==============================] - 167s 17s/step - loss: 1.8245 - accuracy: 0.7844 - val_loss: 1.5511 - val_accuracy: 0.7625
Epoch 6/20
10/10 [==============================] - 161s 16s/step - loss: 1.3253 - accuracy: 0.8125 - val_loss: 2.0526 - val_accuracy: 0.7000
Epoch 7/20
10/10 [==============================] - 156s 16s/step - loss: 1.9675 - accuracy: 0.7812 - val_loss: 3.8209 - val_accuracy: 0.6750
Epoch 

In [ ]:
# Plot the model's training and validation accuracy
plt.plot(history.history['accuracy'], color='blue')
plt.plot(history.history['val_accuracy'], color='red')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Plot the model's training and validation loss
plt.plot(history.history['loss'], color='blue')
plt.plot(history.history['val_loss'], color='red')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Model Testing


In [ ]:
# Provide the path to your locally saved model
model_path = 'Models/PreliminaryTesting/ResNet101_no_rescaling_model.h5'

# Load the model
model.load_weights(model_path)

In [ ]:
model.evaluate(test_it, batch_size=5)

25/25 [==============================] - 133s 5s/step - loss: 4.7554 - accuracy: 0.6725


[4.755430221557617, 0.6725000143051147]

In [ ]:
y_pred = model.predict(test_it, batch_size=5)

In [ ]:
from sklearn.metrics import confusion_matrix

test_Y = test_it.labels
y_pred_binary = np.where(y_pred.flatten() > 0.5, 1, 0)
conf_matrix = confusion_matrix(test_Y,y_pred_binary)

print("Confusion Matrix: ")
print(conf_matrix)